# Waterbomb shell folder and designer

In [1]:
%matplotlib widget

In [2]:
import sympy as sp
#import trimesh
sp.init_printing()
import numpy as np
import matplotlib.pylab as plt
import bmcs_utils.api as bu

In [5]:
from bmcs_shell.api import WBElem4Param as WBElem
from bmcs_shell.folding.wb_shell import WBShell

ModuleNotFoundError: No module named 'bmcs_shell.folding.wb_shell'

In [ ]:
data = dict(alpha=1, a=500, b=900, c=400, n_phi_plus=2, n_x_plus=1, show_wireframe=True)

In [ ]:
wb = WBElem(**data)
ws = WBShell(wb_cell=wb, **data)

In [ ]:
ws.interact()

In [7]:
I_m = ws.I_CDij[-1,:,-1,:]
I_m

array([[11, 10]])

In [9]:
_, idx_remap = ws.unique_node_map
J_m = idx_remap[I_m]

In [10]:
ws.X_Ia[J_m.flatten()].T

array([[-26.48362767, -16.48362767, -12.16120922,  -2.16120922,
          2.16120922,  12.16120922,  16.48362767,  26.48362767],
       [-45.14318848, -45.14318848, -45.14318848, -45.14318848,
        -45.14318848, -45.14318848, -45.14318848, -45.14318848],
       [-18.05472565, -18.05472565, -18.05472565, -18.05472565,
        -18.05472565, -18.05472565, -18.05472565, -18.05472565]])

# Node map to allow for hierarchical sliding

In [13]:
n_cells, n_ic, n_id, _, x_cell_idx, _, y_cell_idx = ws.cell_map
x_cell_idx, y_cell_idx

(array([0, 2]), array([ 0,  2,  4,  6,  8, 10]))

In [14]:
x_idx, y_idx = x_cell_idx / 2, y_cell_idx / 2
x_idx, y_idx

(array([0., 1.]), array([0., 1., 2., 3., 4., 5.]))

In [15]:
n_x_, n_y_ = len(x_idx), len(y_idx) 

In [16]:
I_cell_offset = (n_ic + np.arange(n_x_*n_y_).reshape(n_x_, n_y_)) * ws.wb_cell.n_I
I_cell_offset

array([[ 35,  42,  49,  56,  63,  70],
       [ 77,  84,  91,  98, 105, 112]])

In [17]:
ws.wb_cell.I_boundary[np.newaxis,np.newaxis,:,:]

array([[[[2, 1],
         [6, 5],
         [4, 3]]]])

In [18]:
I_CDij_map = I_cell_offset.T[:,:,np.newaxis, np.newaxis] + ws.wb_cell.I_boundary[np.newaxis,np.newaxis,:,:]

In [19]:
I_CDij_map[-1,:,-1,:]

array([[ 74,  73],
       [116, 115]])

# Find duplicate nodes and merge them to a single one

In [20]:
X_Ia = np.array([[2,1,0],
                 [3,8,9],
                 [0,0,0],
                 [5,3,9],
                 [1,0,1],
                 [0,0,0],
                 [1,0,1],
                 [0,0,0]],dtype=np.float_)

In [21]:
# reshape the coordinates in array of segments to the shape (n_N, n_D
x_0 = X_Ia

In [22]:
# construct distance vectors between every pair of nodes
x_x_0 = x_0[:, np.newaxis, :] - x_0[np.newaxis, :, :]

In [23]:
# calculate the distance between every pair of nodes
dist_0 = np.sqrt(np.einsum('...i,...i', x_x_0, x_x_0))

In [24]:
# identify those at the same location
zero_dist = dist_0 < ws.node_match_threshold
zero_dist

array([[ True, False, False, False, False, False, False, False],
       [False,  True, False, False, False, False, False, False],
       [False, False,  True, False, False,  True, False,  True],
       [False, False, False,  True, False, False, False, False],
       [False, False, False, False,  True, False,  True, False],
       [False, False,  True, False, False,  True, False,  True],
       [False, False, False, False,  True, False,  True, False],
       [False, False,  True, False, False,  True, False,  True]])

In [25]:
# get their indices
i_idx, j_idx = np.where(zero_dist)

In [26]:
# take only the upper triangle indices
upper_triangle = i_idx < j_idx

In [27]:
idx_multi, idx_delete = i_idx[upper_triangle], j_idx[upper_triangle]

In [28]:
# construct a boolean array with True at valid and False at deleted
# indices
idx_unique = np.ones((len(x_0),), dtype='bool')
idx_unique[idx_delete] = False

In [29]:
idx_multi, idx_delete

(array([2, 2, 4, 5]), array([5, 7, 6, 7]))

In [30]:
idx_keep = np.ones((len(x_0),), dtype=np.bool_)
idx_keep[idx_delete] = False

In [31]:
idx_delete, idx_multi

(array([5, 7, 6, 7]), array([2, 2, 4, 5]))

In [32]:
ij_map = np.ones_like(dist_0, dtype=np.int_) + len(x_0)
ij_map

array([[9, 9, 9, 9, 9, 9, 9, 9],
       [9, 9, 9, 9, 9, 9, 9, 9],
       [9, 9, 9, 9, 9, 9, 9, 9],
       [9, 9, 9, 9, 9, 9, 9, 9],
       [9, 9, 9, 9, 9, 9, 9, 9],
       [9, 9, 9, 9, 9, 9, 9, 9],
       [9, 9, 9, 9, 9, 9, 9, 9],
       [9, 9, 9, 9, 9, 9, 9, 9]])

In [33]:
i_ = np.arange(len(x_0))
idx_row = i_[idx_keep]

In [34]:
ij_map[idx_keep,idx_keep] = np.arange(len(idx_row))
ij_map

array([[0, 9, 9, 9, 9, 9, 9, 9],
       [9, 1, 9, 9, 9, 9, 9, 9],
       [9, 9, 2, 9, 9, 9, 9, 9],
       [9, 9, 9, 3, 9, 9, 9, 9],
       [9, 9, 9, 9, 4, 9, 9, 9],
       [9, 9, 9, 9, 9, 9, 9, 9],
       [9, 9, 9, 9, 9, 9, 9, 9],
       [9, 9, 9, 9, 9, 9, 9, 9]])

In [35]:
ij_map[i_idx, j_idx] = ij_map[i_idx, i_idx]
ij_map

array([[0, 9, 9, 9, 9, 9, 9, 9],
       [9, 1, 9, 9, 9, 9, 9, 9],
       [9, 9, 2, 9, 9, 2, 9, 2],
       [9, 9, 9, 3, 9, 9, 9, 9],
       [9, 9, 9, 9, 4, 9, 4, 9],
       [9, 9, 9, 9, 9, 9, 9, 9],
       [9, 9, 9, 9, 9, 9, 9, 9],
       [9, 9, 9, 9, 9, 9, 9, 9]])

In [36]:
idx_remap = np.min(ij_map,axis=0)
idx_remap

array([0, 1, 2, 3, 4, 2, 4, 2])

## Sketch of the applied generation method / concept

Sketch the discretization into the cells the cell midpoints 
cover a cylindric surface as a structured grid. 

Midpoints are arranged at the surface of the derived cylinder
parameters which depend on the folding angle $\alpha$ and $a, b, c$
parameters of the waterbomd cell. Then, a reference cell 
in an intermediate state of folding is broadcasted 
to the prepared grid of midpoints on the cylinders. 

In [39]:
n_phi_plus = 2
delta_phi = 0.3
phi_range = np.arange(-(n_phi_plus-1), n_phi_plus) * delta_phi
r_0 = -10

In [40]:
phi_range

array([-0.3,  0. ,  0.3])

In [41]:
X_phi_range = np.array([np.fabs(r_0) * np.sin(phi_range),
                 np.fabs(r_0) * np.cos(phi_range) + r_0]).T

In [42]:
fix, ax = plt.subplots(1,1)
ax.plot(*X_phi_range.T);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [44]:
n_x_plus = 2
delta_x = 0.5
X_x_range = np.arange(-(n_x_plus-1), n_x_plus) * delta_x
X_x_range

array([-0.5,  0. ,  0.5])

In [45]:
X_x_range[:, np.newaxis, np.newaxis]

array([[[-0.5]],

       [[ 0. ]],

       [[ 0.5]]])

In [46]:
X_phi_range[np.newaxis,:,:]

array([[[-2.95520207, -0.44663511],
        [ 0.        ,  0.        ],
        [ 2.95520207, -0.44663511]]])

In [47]:
n_idx_x = len(X_x_range)
n_idx_phi = len(X_phi_range)
idx_x = np.arange(n_idx_x)
idx_phi = np.arange(n_idx_phi)
idx_phi

array([0, 1, 2])

In [48]:
1+1 % 2

In [50]:
idx_x_ic = idx_x[(n_idx_x)%2::2]
idx_x_id = idx_x[(n_idx_x+1)%2::2]
idx_phi_ic = idx_phi[(n_idx_phi)%2::2]
idx_phi_id = idx_phi[(n_idx_phi+1)%2::2]

idx_x_ic, idx_phi_ic, idx_x_id, idx_phi_id

(array([1]), array([1]), array([0, 2]), array([0, 2]))

In [51]:
idx_map_ic = np.meshgrid(idx_x_ic, idx_phi_ic)
idx_map_id = np.meshgrid(idx_x_id, idx_phi_id)

In [104]:
idx_map_ic

[array([[1]]), array([[1]])]

In [55]:
X_C_a = np.c_[ X_x_range[idx_map_ic[0]].reshape(-1,1), X_phi_range[idx_map_ic[1]].reshape(-1,2) ]
X_D_a = np.c_[ X_x_range[idx_map_id[0]].reshape(-1,1), X_phi_range[idx_map_id[1]].reshape(-1,2) ]

In [56]:
X_C_a

array([[0., 0., 0.]])

In [57]:
X_D_a

array([[-0.5       , -2.95520207, -0.44663511],
       [ 0.5       , -2.95520207, -0.44663511],
       [-0.5       ,  2.95520207, -0.44663511],
       [ 0.5       ,  2.95520207, -0.44663511]])